In [321]:
import numpy as np
import pandas as pd
import tensorflow as tf


In [318]:
def sample_to_7dsignature(*args):
    """
    Takes a CSV file name (without extension) having samples of device as input.
    The column names must be as : V1,C1,A1,PF1,AP1,RP1,F1,T
    Converts it to a dataset of signatures of duration 1 min.
    Exports the modified CSV in seconds frequency, and signature format in separate files as: 
        1. (filename_seconds.csv) 
        2. (filename_7d.npy).
        3. 7D numpy array
        4. labels
    Returns 1. the 7-D signature of device, and the 2. appropriate labels in a pandas series format. 
    
    """
    import csv
    import numpy as np
    import pandas as pd
    
    devices = []
    for x in args:
        devices.append(x)
    for fn in devices:
        df = pd.read_csv(f'{fn}.csv')
        df['T'] = pd.DatetimeIndex(df['T'],tz = 'Asia/Kolkata')
        df = df.set_index('T')
        df.resample('S').mean()
        df.to_csv(f'{fn}_seconds.csv',index= False)

    
        with open(f'{fn}_seconds.csv', 'r') as fp:
            next(fp) # ignore column names
            np_v  =np.array([])
            np_c  =np.array([])
            np_a  =np.array([])
            np_pf =np.array([])
            np_ap =np.array([])
            np_rp =np.array([])
            np_f  =np.array([])
            params = [np_v,np_c,np_a,np_pf,np_ap,np_rp,np_f]  

            for i, line in enumerate(fp):
                line  = line[:-2]  #remove \n from end of string.
                line = np.array(line.split(sep = ','))  # array of string values
                line = line.astype(np.float)    # array of float values v,c,a,pf,ap,rp,f
    #             print(line) 
    #             print(line.size)  7
    #             print(line.shape) 7,
                for idx, val in enumerate(line):
                    #i will give us the parameter name ,  x will give us the parameter value
                # not able to reduce code due to some issue with numpy array not being updated
                    if idx == 0:
                        np_v = np.append(np_v,val)
                    elif idx==1:
                        np_c = np.append(np_c,val)
                    elif idx==2:
                        np_a = np.append(np_a,val)
                    elif idx==3:
                        np_pf = np.append(np_pf,val)
                    elif idx==4:
                        np_ap = np.append(np_ap,val)
                    elif idx==5:
                        np_rp = np.append(np_rp,val)
                    elif idx==6:
                        np_f = np.append(np_f,val) # append values to corresponding arrays 
    #         print(np_v,np_c,np_a,np_pf,np_ap,np_rp,np_f)  #correct
    #         print(np_v.size) #5559



            extra_samples = (np_v.size)%60  # a signature is of length 60, thus delete extra samples
    #         print(np_v.shape) (5559,)

            if extra_samples !=0:
                np_v = np.delete(np_v,np.s_[-extra_samples:])
                np_c = np.delete(np_c,np.s_[-extra_samples:])
                np_a = np.delete(np_a,np.s_[-extra_samples:])
                np_pf = np.delete(np_pf,np.s_[-extra_samples:])
                np_ap = np.delete(np_ap,np.s_[-extra_samples:])
                np_rp = np.delete(np_rp,np.s_[-extra_samples:])
                np_f = np.delete(np_f,np.s_[-extra_samples:])



            samples = np_v.size
            no_signatures = samples//60  # no of signatures per device collected

            np_v = np_v.reshape((no_signatures),60)  #convert samples to a stream of 1 minute
            np_c = np_c.reshape((no_signatures),60)
            np_a = np_a.reshape((no_signatures),60)
            np_pf = np_pf.reshape((no_signatures),60)
            np_ap = np_ap.reshape((no_signatures),60)
            np_rp = np_rp.reshape((no_signatures),60)
            np_f = np_f.reshape((no_signatures),60)
    #         print(np_v.shape)  (92, 60)


            li = [] #this will hold all the signatures of a device 
    #         print(no_signatures)  #92
            for x in range(no_signatures):
                # signature_array will be 7 dimensional array with a length of 1 row x 60 columns
                signature_array = np.array([
                                            [np_v[x]],
                                            [np_c[x]],
                                            [np_a[x]],
                                            [np_pf[x]],
                                            [np_ap[x]],
                                            [np_rp[x]],
                                            [np_f[x]],
                                          ])
    #             print(signature_array.shape)  #(7, 1, 60)
                li.append(signature_array)
            li = np.array(li)
    #         print(li.shape) #(92, 7, 1, 60)
            no_rows = li.shape[0]
    #         print(no_rows) #92
            label_series = pd.Series([],dtype='string')
            for x in range (no_rows):
                label_series[x] = fn
            np.save(f'{fn}',li)
            label_series.to_csv(f'{fn}_labels.csv',index=False)
    return 

In [310]:
labels = pd.Series([],dtype = 'string')

In [320]:
sample_to_7dsignature('3dprinter','ac','fridge','wallfan')

In [355]:
arr = np.load('3dprinter.npy')
arr.shape
        

(92, 7, 1, 60)

In [356]:
brr = np.load('ac.npy')
brr.shape

(99, 7, 1, 60)

In [344]:
crr = np.append(arr,brr,axis = 0)

In [345]:
crr.shape

(191, 7, 1, 60)

In [353]:
crr[91][1][0][0]

0.41

In [370]:
a = np.array([[1,2,3],[4,5,6],[13,14,15]])
b = np.array([[7,8,9],[10,11,12]])

In [371]:
a.shape

(3, 3)

In [373]:
b.shape

(2, 3)

In [372]:
c= np.array([])
c = np.append(c,a)
c= np.append(c,b)
c.shape

(15,)

In [374]:
c = c.reshape((5,3))

In [375]:
c

array([[ 1.,  2.,  3.],
       [ 4.,  5.,  6.],
       [13., 14., 15.],
       [ 7.,  8.,  9.],
       [10., 11., 12.]])

In [394]:
def get_xy(*args):
    devices = []
    for x in args:
        devices.append(x)
    size = 0
    files = np.array([])
    labels = np.array([])
    for fn in devices:
        frr = np.load(f'{fn}.npy')
        files = np.append(files,frr)
        lrr = pd.read_csv(f'{fn}_labels.csv').to_numpy()
        labels = np.append(labels,lrr)
        s = frr.shape[0]
        size += s
#     print(size) #379
    files = files.reshape((size,7,1,60))
#     print(files.shape,labels.shape) #(379, 7, 1, 60) (379,)
    return files,labels        

In [413]:
files,labels = get_xy('3dprinter','ac','fridge','wallfan')

92
191
284
379
(379, 7, 1, 60) (379,)


In [414]:
unique_devices = np.unique(labels)
unique_devices

array(['3dprinter', 'ac', 'fridge', 'wallfan'], dtype=object)

In [415]:
y[0]==unique_devices

array([ True, False, False, False])

In [417]:
# Turn every label into a boolean array
boolean_labels = [label == np.array(unique_devices) for label in labels]
boolean_labels

[array([ True, False, False, False]),
 array([ True, False, False, False]),
 array([ True, False, False, False]),
 array([ True, False, False, False]),
 array([ True, False, False, False]),
 array([ True, False, False, False]),
 array([ True, False, False, False]),
 array([ True, False, False, False]),
 array([ True, False, False, False]),
 array([ True, False, False, False]),
 array([ True, False, False, False]),
 array([ True, False, False, False]),
 array([ True, False, False, False]),
 array([ True, False, False, False]),
 array([ True, False, False, False]),
 array([ True, False, False, False]),
 array([ True, False, False, False]),
 array([ True, False, False, False]),
 array([ True, False, False, False]),
 array([ True, False, False, False]),
 array([ True, False, False, False]),
 array([ True, False, False, False]),
 array([ True, False, False, False]),
 array([ True, False, False, False]),
 array([ True, False, False, False]),
 array([ True, False, False, False]),
 array([ Tru

In [418]:
# Setup X & y variables
X = files
y = boolean_labels

In [419]:
# Import train_test_split from Scikit-Learn
from sklearn.model_selection import train_test_split

# Split them into training and validation using NUM_IMAGES 
X_train, X_val, y_train, y_val = train_test_split(X,
                                                  y, 
                                                  test_size=0.2,
                                                  random_state=42)

len(X_train), len(y_train), len(X_val), len(y_val)

(303, 303, 76, 76)

In [422]:
# Check out the training data ( file  and labels)
X_train[0], y_train[0]

(array([[[ 2.30540e+02,  2.30500e+02,  2.30490e+02,  2.30390e+02,
           2.30180e+02,  2.30490e+02,  2.29980e+02,  2.29630e+02,
           2.29910e+02,  2.30000e+02,  2.30030e+02,  2.30430e+02,
           2.30070e+02,  2.30560e+02,  2.30310e+02,  2.29880e+02,
           2.31380e+02,  2.31080e+02,  2.31390e+02,  2.31290e+02,
           2.31460e+02,  2.31480e+02,  2.31430e+02,  2.31250e+02,
           2.30760e+02,  2.31390e+02,  2.31470e+02,  2.31500e+02,
           2.31490e+02,  2.31000e+02,  2.30780e+02,  2.31010e+02,
           2.31090e+02,  2.30960e+02,  2.31520e+02,  2.31090e+02,
           2.31500e+02,  2.31500e+02,  2.31540e+02,  2.30900e+02,
           2.31540e+02,  2.31610e+02,  2.31320e+02,  2.31090e+02,
           2.31160e+02,  2.31470e+02,  2.31490e+02,  2.31340e+02,
           2.31550e+02,  2.31550e+02,  2.31530e+02,  2.30390e+02,
           2.31100e+02,  2.31270e+02,  2.31260e+02,  2.31270e+02,
           2.31290e+02,  2.31380e+02,  2.31730e+02,  2.31460e+02]],
 
      

In [427]:
# Define the batch size, 32 is a good default
BATCH_SIZE = 32

# Create a function to turn data into batches
def create_data_batches(x, y=None, batch_size=BATCH_SIZE, valid_data=False, test_data=False):
    """
      Creates batches of data out of  (x) and label (y) pairs.
      Shuffles the data if it's training data but doesn't shuffle it if it's validation data.
      Also accepts test data as input (no labels).
    """
  # If the data is a test dataset, we probably don't have labels
    if test_data:
        print("Creating test data batches...")
        data = tf.data.Dataset.from_tensor_slices((tf.constant(x))) # only files
        data_batch = data.batch(BATCH_SIZE)
        return data_batch
  
  # If the data if a valid dataset, we don't need to shuffle it
    elif valid_data:
        print("Creating validation data batches...")
        data = tf.data.Dataset.from_tensor_slices((tf.constant(x), # filepaths
                                                   tf.constant(y))) # labels
        data_batch = data.batch(BATCH_SIZE)
        return data_batch

    else:
        # If the data is a training dataset, we shuffle it
        print("Creating training data batches...")
        # Turn filepaths and labels into Tensors
        data = tf.data.Dataset.from_tensor_slices((tf.constant(x), # filepaths
                                                  tf.constant(y))) # labels
    
        # Shuffling pathnames and labels before mapping image processor function is faster than shuffling images
        data = data.shuffle(buffer_size=len(x))

        
        # Turn the data into batches
        data_batch = data.batch(BATCH_SIZE)
    return data_batch

In [428]:
# Create training and validation data batches
train_data = create_data_batches(X_train, y_train)
val_data = create_data_batches(X_val, y_val, valid_data=True)

Creating training data batches...
Creating validation data batches...


In [429]:
# Check out the different attributes of our data batches
train_data.element_spec, val_data.element_spec

((TensorSpec(shape=(None, 7, 1, 60), dtype=tf.float64, name=None),
  TensorSpec(shape=(None, 4), dtype=tf.bool, name=None)),
 (TensorSpec(shape=(None, 7, 1, 60), dtype=tf.float64, name=None),
  TensorSpec(shape=(None, 4), dtype=tf.bool, name=None)))

In [501]:
# Setup input shape to the model
INPUT_SHAPE = [None, 7, 1, 60] # batch, rows, columns, features

# Setup output shape of the model
OUTPUT_SHAPE = len(unique_devices) # number of unique labels



# installing package in jupyter

In [502]:
# import sys
# !pip install --upgrade tensorflow_hub



In [516]:
from sklearn.model_selection import RepeatedKFold
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score
from tensorflow import keras
from tensorflow.keras import layers

In [552]:
# Create a function which builds a Keras model
def create_model(input_shape=INPUT_SHAPE, output_shape=OUTPUT_SHAPE):
    print("Building model with")
    
    results = np.array([])
    np.random.seed(42)
  # Setup the model layers
    model = keras.Sequential()
    model.add(layers.Dense(60, activation="relu", input_shape=(7,1,60), kernel_initializer='he_uniform'))
#     model.add(layers.Dense(4, activation='sigmoid'))
    model.add(tf.keras.layers.Flatten()) 
    model.add(Dense(64))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(Dense(4, activation='sigmoid'))

  # Compile the model
    model.compile(loss='binary_crossentropy', optimizer='adam')
    
    model.build(INPUT_SHAPE) # Let the model know what kind of inputs it'll be getting

    return model

In [553]:
# Create a model and check its details
model = create_model()
model.summary()

Building model with
Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_46 (Dense)             (None, 7, 1, 60)          3660      
_________________________________________________________________
flatten_4 (Flatten)          (None, 420)               0         
_________________________________________________________________
dense_47 (Dense)             (None, 64)                26944     
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_48 (Dense)             (None, 4)                 260       
Total params: 30,864
Trainable params: 30,864
Non-trainable params: 0
_________________________________________________________________


In [554]:
model.fit(x = train_data , validation_data = val_data ,verbose=0, epochs=100)

In [555]:
predictions = model.predict(val_data, verbose=1)

3/3 [==============================] - 0s 3ms/step


In [557]:
# First prediction
print(predictions[0])
print(f"Max value (probability of prediction): {np.max(predictions[0])}") # the max probability value predicted by the model
print(f"Sum: {np.sum(predictions[0])}") # because we used softmax activation in our model, this will be close to 1
print(f"Max index: {np.argmax(predictions[0])}") # the index of where the max value in predictions[0] occurs
print(f"Predicted label: {unique_devices[np.argmax(predictions[0])]}") # the predicted label

[3.444146e-36 0.000000e+00 0.000000e+00 1.000000e+00]
Max value (probability of prediction): 1.0
Sum: 1.0
Max index: 3
Predicted label: wallfan


In [558]:
# Turn prediction probabilities into their respective label (easier to understand)
def get_pred_label(prediction_probabilities):
  """
  Turns an array of prediction probabilities into a label.
  """
  return unique_devices[np.argmax(prediction_probabilities)]

# Get a predicted label based on an array of prediction probabilities
pred_label = get_pred_label(predictions[0])
pred_label

'wallfan'

In [560]:
# Create a function to unbatch a batched dataset
def unbatchify(data):
    """
  Takes a batched dataset of (file, label) Tensors and returns separate arrays
  of images and labels.
    """
    files = []
    labels = []
  # Loop through unbatched data
    for file, label in data.unbatch().as_numpy_iterator():
        files.append(file)
        labels.append(unique_devices[np.argmax(label)])
    return files, labels

# Unbatchify the validation data
val_files, val_labels = unbatchify(val_data)
val_files[0], val_labels[0]

(array([[[2.9645e+02, 2.9578e+02, 2.9641e+02, 2.9578e+02, 2.9653e+02,
          2.9589e+02, 2.9641e+02, 2.9608e+02, 2.9534e+02, 2.9625e+02,
          2.9505e+02, 2.9659e+02, 2.9624e+02, 2.9671e+02, 2.9681e+02,
          2.9687e+02, 2.9707e+02, 2.9627e+02, 2.9687e+02, 2.9614e+02,
          2.9634e+02, 2.9651e+02, 2.9590e+02, 2.9648e+02, 2.9594e+02,
          2.9676e+02, 2.9681e+02, 2.9618e+02, 2.9642e+02, 2.9601e+02,
          2.9682e+02, 2.9617e+02, 2.9644e+02, 2.9602e+02, 2.9545e+02,
          2.9639e+02, 2.9589e+02, 2.9655e+02, 2.9667e+02, 2.9669e+02,
          2.9677e+02, 2.9678e+02, 2.9682e+02, 2.9623e+02, 2.9644e+02,
          2.9629e+02, 2.9660e+02, 2.9616e+02, 2.9620e+02, 2.9604e+02,
          2.9771e+02, 2.9833e+02, 2.9654e+02, 2.9643e+02, 2.9643e+02,
          2.9688e+02, 2.9684e+02, 2.9576e+02, 2.9640e+02, 2.9590e+02]],
 
        [[2.8000e-01, 2.8000e-01, 2.8000e-01, 2.8000e-01, 2.8000e-01,
          2.8000e-01, 2.8000e-01, 2.8000e-01, 2.8000e-01, 2.8000e-01,
          2.8000

In [570]:
def plot_pred(prediction_probabilities, labels, files, n=1):
    """
  View the prediction, ground truth label and image for sample n.
    """
    pred_prob, true_label, file = prediction_probabilities[n], labels[n], files[n]
  
  # Get the pred label
    pred_label = get_pred_label(pred_prob)
  


  # Change the color of the title depending on if the prediction is right or wrong
    if pred_label == true_label:
        val = "True"
    else:
        val = "False"

    df = pd.DataFrame({'Predicted':pred_label,'Actual': true_label,
                    'Assumed':val},index = [n])
    return df

In [571]:
# View an example prediction, original image and truth label
plot_pred(prediction_probabilities=predictions,
          labels=val_labels,
          files=val_files)

,Predicted,Actual,Assumed
1,fridge,fridge,True


In [572]:
pred_df = pd.DataFrame()

for x in range(76):
    data_frame = plot_pred(prediction_probabilities=predictions,
        labels=val_labels,
        files=val_files,n=x)
    pred_df = pred_df.append(data_frame)

In [573]:
pred_df

,Predicted,Actual,Assumed
0,wallfan,wallfan,True
1,fridge,fridge,True
2,wallfan,wallfan,True
3,ac,ac,True
4,3dprinter,3dprinter,True
...,...,...,...
71,fridge,fridge,True
72,ac,ac,True
73,wallfan,wallfan,True
74,ac,ac,True


In [574]:
pred_df.head(60)

,Predicted,Actual,Assumed
0,wallfan,wallfan,True
1,fridge,fridge,True
2,wallfan,wallfan,True
3,ac,ac,True
4,3dprinter,3dprinter,True
5,ac,ac,True
6,wallfan,wallfan,True
7,3dprinter,3dprinter,True
8,wallfan,wallfan,True
9,ac,ac,True


In [569]:
pred_df.tail(60)

,Predicted,Actual,Assumed
16,wallfan,wallfan,True
17,ac,ac,True
18,wallfan,wallfan,True
19,ac,ac,True
20,3dprinter,3dprinter,True
21,ac,ac,True
22,ac,ac,True
23,3dprinter,3dprinter,True
24,3dprinter,3dprinter,True
25,3dprinter,fridge,False


In [575]:
pred_df['Assumed'].value_counts()

True     75
False     1
Name: Assumed, dtype: int64

In [576]:
model.evaluate(val_data)

3/3 [==============================] - 0s 5ms/step - loss: 0.5854


0.5853524208068848